## Controlas Fechas de Actualización de las Tablas

- monitorear_actualizacion_tablas.py
- Objetivo: Verificar que las tablas críticas del esquema `src` estén actualizadas con datos recientes.
- Autor: [Zeetrex]
- Fecha: [Auto-generado]

In [4]:
import psycopg2 as pg2
from datetime import date
import time
import logging
import os
import sys
from dotenv import dotenv_values
import pandas as pd

# Configurar logging
logging.basicConfig(
    filename='./logs/monitoreo_tablas.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

ENV_PATH = os.environ.get("ETL_ENV_PATH", "C:/ETL/ETL_DIARCO/.env")  # Toma Producción si está definido, o la ruta por defecto E:\ETL\ETL_DIARCO\.env
# Verificar si el archivo .env existe
if not os.path.exists(ENV_PATH):
    print(f"El archivo .env no existe en la ruta: {ENV_PATH}")
    print(f"Directorio actual: {os.getcwd()}")
    sys.exit(1)
    
secrets = dotenv_values(ENV_PATH)

def Open_Diarco_Data(): 
    conn_str = f"dbname={secrets['PG_DB']} user={secrets['PG_USER']} password={secrets['PG_PASSWORD']} host={secrets['PG_HOST']} port={secrets['PG_PORT']}"
    #print (conn_str)
    for i in range(5):
        try:    
            conn = pg2.connect(conn_str)
            return conn
        except Exception as e:
            print(f'Error en la conexión: {e}')
            time.sleep(5)
    return None  # Retorna None si todos los intentos fallan


In [2]:
# Tablas a controlar
tablas_originales_a_controlar = [
    "sucursales_excluidas",  
    "t055_articulos_condcompra_costos", "historico_stock_sucursal",
    "t085_articulos_ean_edi", "t702_est_vtas_por_articulo", "t710_estadis_stock",
    "t114_rubros", "t117_compradores", 
     "base_forecast_oc_demoradas", 
    "base_forecast_stock", "base_forecast_ventas", 
    "m_91_sucursales", "base_forecast_precios", 
    
    "t055_articulos_param_stock",
    "t052_articulos_proveedor",  "t055_lead_time_b2_sucursales"
]
# Contiene campo fecha_extraccion
tablas_a_controlar = [
    "base_forecast_articulos", "base_productos_vigentes", "base_stock_sucursal",
    "m_3_articulos", "t020_proveedor_gestion_compra", "t050_articulos", "t051_articulos_sucursal", 
    "t020_proveedor", "t060_stock", "t080_oc_cabe", "t081_oc_deta", "t100_empresa_suc",
    "t702_est_vtas_por_articulo_dbarrio", "t710_estadis_oferta_folder", "t710_estadis_precios",
    "t710_estadis_reposicion"
]


In [8]:
def controlar_tablas(conn):
    cur = conn.cursor()
    for tabla in tablas_a_controlar:
        try:
            query = f"""
                SELECT
                    MAX(fecha_extraccion) AS ultima_fecha,
                    COUNT(*) AS cantidad
                FROM src.{tabla}
                WHERE fecha_extraccion::date = (
                    SELECT MAX(fecha_extraccion::date)
                    FROM src.{tabla}
                )
            """
            cur.execute(query)
            resultado = cur.fetchone()
            ultima_fecha, cantidad = resultado

            cur.execute("""
                INSERT INTO src.monitoreo_estado_tablas (tabla, ultima_fecha_extraccion, cantidad_registros)
                VALUES (%s, %s, %s)
                ON CONFLICT (tabla) DO UPDATE SET
                    ultima_fecha_extraccion = EXCLUDED.ultima_fecha_extraccion,
                    cantidad_registros = EXCLUDED.cantidad_registros,
                    fecha_control = NOW()
            """, (tabla, ultima_fecha, cantidad))
            logging.info(f"✔ Tabla {tabla}: {cantidad} registros con fecha {ultima_fecha}")
        except Exception as e:
            logging.error(f"❌ Error en la tabla {tabla}: {str(e)}")
            continue

    conn.commit()
    cur.close()

In [7]:
import pandas as pd

columns = ["tabla", "ultima_fecha_extraccion", "cantidad_registros"]
df_control = pd.DataFrame(columns=columns)

conn = Open_Diarco_Data()
cur = conn.cursor()

for tabla in tablas_a_controlar:
    try:
        query = f"""
            SELECT
                MAX(fecha_extraccion) AS ultima_fecha,
                COUNT(*) AS cantidad
            FROM src.{tabla}
            WHERE fecha_extraccion::date = (
                SELECT MAX(fecha_extraccion::date)
                FROM src.{tabla}
        """
        cur.execute(query)
        resultado = cur.fetchone()
        ultima_fecha, cantidad = resultado

       # Insertar en DataFrame
        df_control = df_control.append({
            "tabla": tabla,
            "ultima_fecha_extraccion": ultima_fecha,
            "cantidad_registros": cantidad
        }, ignore_index=True)

    except Exception as e:
        logging.error(f"❌ Error en la tabla {tabla}: {str(e)}")
        continue

conn.commit()
cur.close()

In [9]:

if __name__ == "__main__":
    try:
        conn = Open_Diarco_Data()
        controlar_tablas(conn)
        conn.close()
        logging.info("✅ Monitoreo completado correctamente.")
    except Exception as e:
        logging.critical(f"🚨 Fallo de conexión o ejecución: {str(e)}")
